In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
drive.mount('/content/gdrive')
bengali_news_after_preprocessing = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/Bengali_Text_after_preprocessing.pkl')
from sklearn.externals import joblib
filename = '/content/gdrive/My Drive/Projects/Bengali Text Classification/fastText_Bangla_content_full.sav'
loaded_model = joblib.load(filename)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
import keras.backend as K
import numpy as np
number_of_sample, max_number_of_words, word_vector_size = 40000, 100, 32
temp = bengali_news_after_preprocessing.loc[:number_of_sample-1,:max_number_of_words-1]

Using TensorFlow backend.


In [0]:
temp = temp.replace(['ঘস', 'ফগ', 'ঝবঃ', 'ঋন', 'ঊঘ', '\u09e4', 'ওৎ', 'গথ', 'খঢ', 'ঝ’', 'ং', 'ঔ', 'ডড', 'গঘ','ঐব','ওঃ’ং','ুী','খষ','ি'], None)
X = np.zeros((number_of_sample, max_number_of_words, word_vector_size), dtype=K.floatx())
for i in temp.index:
  X[i,:,:] = loaded_model.wv[temp.loc[i,:]]

In [0]:
bengali_news = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/40k_bangla_newspaper_article.p')
bengali_news_dataframe = pd.DataFrame(bengali_news)
y = bengali_news_dataframe['category']

In [0]:
from sklearn import preprocessing
import keras
import numpy as np
le = preprocessing.LabelEncoder()
le.fit(y)
enc = le.transform(y)
y = keras.utils.to_categorical(enc)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle = True, test_size=0.125)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, shuffle = True, test_size=0.20)

In [7]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard


model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same',
                 input_shape=(max_number_of_words, word_vector_size)))
model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=3))

model.add(Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.3)))

model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))

model.add(Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

W0614 12:52:02.439566 139889359779712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0614 12:52:02.458703 139889359779712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0614 12:52:02.461921 139889359779712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0614 12:52:02.528251 139889359779712 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0614 12:52:02.960746 139889359779712 deprecation_wrapp

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 33, 32)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               591872    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

In [9]:
history = model.fit(X_train, y_train, batch_size= 500, shuffle=True, epochs= 100, validation_data=(X_val, y_val))

W0614 12:52:04.171535 139889359779712 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 35000 samples, validate on 4000 samples
Epoch 1/100
35000/35000 [==============================] - 21s 600us/step - loss: 2.4041 - acc: 0.2276 - val_loss: 1.9476 - val_acc: 0.3095
Epoch 2/100
35000/35000 [==============================] - 16s 462us/step - loss: 1.9944 - acc: 0.2973 - val_loss: 1.8544 - val_acc: 0.4815
Epoch 3/100
35000/35000 [==============================] - 16s 461us/step - loss: 1.8398 - acc: 0.4225 - val_loss: 1.6712 - val_acc: 0.4970
Epoch 4/100
35000/35000 [==============================] - 16s 463us/step - loss: 1.6910 - acc: 0.4790 - val_loss: 1.5533 - val_acc: 0.5057
Epoch 5/100
35000/35000 [==============================] - 16s 461us/step - loss: 1.5937 - acc: 0.4930 - val_loss: 1.4571 - val_acc: 0.5345
Epoch 6/100
35000/35000 [==============================] - 16s 462us/step - loss: 1.4871 - acc: 0.5262 - val_loss: 1.3351 - val_acc: 0.6027
Epoch 7/100
35000/35000 [==============================] - 16s 461us/step - loss: 1.3770 - acc: 0.5836 - val_lo

In [0]:
predicts = model.predict(X_test)
import numpy as np
def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [11]:
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_preds))

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

0.843
[[  0   0   0   0   0   1   0   0   0   4   0   0]
 [  0 257   0  12   3   0   4   1   0  26   2   1]
 [  0   1   0   0   0   0   0   0   0   1   0   0]
 [  0   5   0 104   2   0   1   0   0   5   0   3]
 [  0   4   0   1  12   0   0   1   0   1   0   1]
 [  0   2   0   0   0  61   1   0   0   0   0   4]
 [  0   6   0   2   0   2  29   0   0   4   0   0]
 [  0   1   0   0   0   2   0  12   0  12   0   2]
 [  0   1   0   0   0   0   3   0   0   1   0   0]
 [  0  13   0   1   0   2   1   2   0 255   2   2]
 [  0   0   0   0   0   0   0   0   0   1  79   0]
 [  0   4   0   1   0   1   1   1   0   2   0  34]]
                    precision    recall  f1-score   support

art-and-literature       0.00      0.00      0.00         5
        bangladesh       0.87      0.84      0.86       306
       durporobash       0.00      0.00      0.00         2
           economy       0.86      0.87      0.86       120
         education       0.71      0.60      0.65        20
     entertainment  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
accuracy, val_accuracy = np.array(history.history["acc"]), np.array(history.history["val_acc"])
accuracy, val_accuracy = accuracy.reshape(100,1), val_accuracy.reshape(100,1)
accuracies = np.concatenate((accuracy,val_accuracy),axis=1)
np.savetxt('/content/gdrive/My Drive/Projects/Bengali Text Classification/temp.csv',accuracies,delimiter=",")